In [1]:
import gc
import glob

import geopandas as gpd
import momepy as mm

import numpy as np
import pandas as pd
import shapely
from libpysal.graph import Graph, read_parquet
import datetime

In [2]:
from core.generate_buildings import read_region_buildings, process_region_buildings
from core.generate_streets import process_region_streets, process_single_region_streets
from core.generate_elements import process_region_elements, generate_enclosures_representative_points, generate_tess
from core.generate_ngraphs import process_region_graphs
from core.generate_chars import process_single_region_chars
from core.generate_merged_primary_chars import merge_into_primary
from core.generate_clusters import process_single_region_morphotopes

In [3]:
country = "belgium"

In [4]:
regions_buildings_dir = '/data/uscuni-ulce/buildings_in_regions/'
buildings_dir = '/data/uscuni-ulce/processed_data/buildings/'
overture_streets_dir = '/data/uscuni-ulce/overture_streets/'
streets_dir = '/data/uscuni-ulce/processed_data/streets/'
enclosures_dir = '/data/uscuni-ulce/processed_data/enclosures/'
tessellations_dir = '/data/uscuni-ulce/processed_data/tessellations/'
graph_dir = '/data/uscuni-ulce/processed_data/neigh_graphs/'
chars_dir = '/data/uscuni-ulce/processed_data/chars/'
simplfied_buildings_dir = '/data/uscuni-ulce/processed_data/simplified_buildings/'

morphotopes_dir = '/data/uscuni-ulce/processed_data/morphotopes/'
regions_datadir = "/data/uscuni-ulce/"

In [5]:
from core.generate_clusters import process_single_region_morphotopes

In [6]:
 process_single_region_morphotopes(region_id,
            graph_dir,
            buildings_dir,
            streets_dir,
            enclosures_dir,
            tessellations_dir,
            chars_dir, morphotopes_dir)

NameError: name 'region_id' is not defined

## Process multiple regions

In [7]:
regions_to_process = gpd.read_parquet(regions_datadir + "regions/" + f"{country}_regions_hull.parquet").index.values
regions_to_process

array([153512, 153650, 153752, 153804, 154059, 154947, 155610, 155668,
       155971])

In [8]:
for region_id in regions_to_process[:1]:

    print(region_id)
    # # buildings
    # buildings = gpd.read_parquet(regions_buildings_dir + f'buildings_{region_id}.pq')
    # buildings = process_region_buildings(buildings, True, simplification_tolerance=.1, merge_limit=25)
    # buildings.to_parquet(buildings_dir + f"buildings_{region_id}.parquet")

    ## processs streets
    streets = process_region_streets(region_id, overture_streets_dir, buildings_dir)
    streets.to_parquet(streets_dir + f'streets_{region_id}.parquet')

    # elements
    enclosures, tesselations = process_region_elements(buildings_dir, streets_dir, region_id)
    enclosures.to_parquet(enclosures_dir + f"enclosure_{region_id}.parquet")
    tesselations.to_parquet(
        tessellations_dir + f"tessellation_{region_id}.parquet"
    )

    #graphs
    process_region_graphs(
    region_id,
    graph_dir,
    buildings_dir,
    streets_dir,
    enclosures_dir,
    tessellations_dir,
    )

    #chars
    process_single_region_chars(
        region_id,
        graph_dir,
        buildings_dir,
        streets_dir,
        enclosures_dir,
        tessellations_dir,
        chars_dir
    )

    # #primary merge
    merge_into_primary(region_id,
        graph_dir,
        buildings_dir,
        streets_dir,
        enclosures_dir,
        tessellations_dir,
        chars_dir)

    # # morphotope
    process_single_region_morphotopes(region_id,
        graph_dir,
        buildings_dir,
        streets_dir,
        enclosures_dir,
        tessellations_dir,
        chars_dir, morphotopes_dir)
    

153512


/home/martin/dev/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/neatnet/nodes.py:377: UserWarning: Unable to sort modes: '<' not supported between instances of 'Point' and 'Point'
  mode = loop_points.loc[loop_points_ix].geometry.mode()
/home/martin/dev/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/neatnet/nodes.py:385: UserWarning: Unable to sort modes: '<' not supported between instances of 'Point' and 'Point'
  new_mode = loop_points.loc[loop_points_ix].geometry.mode()
/home/martin/dev/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/neatnet/nodes.py:377: UserWarning: Unable to sort modes: '<' not supported between instances of 'Point' and 'Point'
  mode = loop_points.loc[loop_points_ix].geometry.mode()
/home/martin/dev/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/neatnet/nodes.py:385: UserWarning: Unable to sort modes: '<' not supported between instances of 'Point' and 'Point'
  new_mode = loop_points.loc[loop_points_

---- Processing region:  153512 2025-05-22 21:11:15.796640


/home/martin/dev/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/home/martin/dev/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Retrying tesselation with less buildings, potentially changing building data.


...

Built nodes graph knn=1
2025-05-22 22:22:12.405767 ----Processing ------ 153512
Processing streets
Processing enclosures
Processing buildings


/home/martin/dev/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/momepy/functional/_shape.py:879: RuntimeWarning: invalid value encountered in arccos
  angles = np.arccos(cosine_angle)
/home/martin/dev/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/momepy/functional/_shape.py:569: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(90 - degrees[true_angles]))
/home/martin/dev/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/momepy/functional/_shape.py:879: RuntimeWarning: invalid value encountered in arccos
  angles = np.arccos(cosine_angle)
/home/martin/dev/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/momepy/functional/_shape.py:726: RuntimeWarning: Mean of empty slice
  return Series({"mean": np.nanmean(dists), "std": np.nanstd(dists)})
/home/martin/dev/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:2019: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = 

Processing tessellation
Processing region 153512
2025-05-23 02:19:30.942153 ----Processing ------ 153512
--------Generating lag----------
--------Generating morphotopes---------- 75
--------Dropping columns---------- ['stcSAl', 'stbOri', 'stcOri', 'stbCeA', 'ldkAre', 'ldkPer', 'lskCCo', 'lskERI', 'lskCWA', 'ltkOri', 'ltkWNB', 'likWBB', 'likWCe', 'licBAD', 'misBAD', 'ssbCCM', 'ssbCCD']


/home/martin/dev/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/sklearn/preprocessing/_data.py:2829: UserWarning: n_quantiles (1000) is greater than the total number of samples (93). n_quantiles is set to n_samples.
  warnings.warn(
/home/martin/dev/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/sklearn/preprocessing/_data.py:2829: UserWarning: n_quantiles (1000) is greater than the total number of samples (407). n_quantiles is set to n_samples.
  warnings.warn(
/home/martin/dev/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/sklearn/preprocessing/_data.py:2829: UserWarning: n_quantiles (1000) is greater than the total number of samples (927). n_quantiles is set to n_samples.
  warnings.warn(
/home/martin/dev/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/sklearn/preprocessing/_data.py:2829: UserWarning: n_quantiles (1000) is greater than the total number of samples (648). n_quantiles is set to n_samples.
  warnings.warn(
/

--------Generating morphotopes data----------
